# PyCity Schools Analysis - Kolencherry


In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_full = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary


In [2]:
#total number of schools 
school_ct = school_data_full.school_name.value_counts().count()

#total number of students - need to use Student ID because common names 
stu_ct = school_data_full['Student ID'].value_counts().count()

#total budget - need to account for duplicate values 
budg_tot = school_data_full.groupby('school_name').mean()['budget'].sum()

#average math score
avg_math = round(school_data_full.math_score.mean(),2)

#average reading score
avg_read = school_data_full.reading_score.mean()

#percentage of students with passing math score
pass_math = len([x for x in school_data_full.math_score if x >= 70])
pass_math_pct = pass_math/stu_ct*100

#percentage of students with passing reading score
pass_read = len([x for x in school_data_full.reading_score if x >= 70])
pass_read_pct = pass_read/stu_ct*100

#percentage of students who passed math and reading
mask = (school_data_full.reading_score >=70) & (school_data_full.math_score>=70)
pass_both = school_data_full.student_name.loc[mask].count()
pass_both_pct = pass_both/stu_ct*100

In [3]:
district_summary = pd.DataFrame({"Total Schools": [school_ct],
             "Total Student": [stu_ct],
             "Total Budget": [budg_tot],
             "Average Math Score": [avg_math],
             "Average Reading Score": [avg_read],
             "% Passing Math": [pass_math_pct],
             "% Passing Reading": [pass_read_pct],
             "% Passing Math and Reading": [pass_both_pct]})

district_summary

,Total Schools,Total Student,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Math and Reading
0,15,39170,24649428.0,78.99,81.87784,74.980853,85.805463,65.172326


## School Summary

In [4]:
#return school type for each school
sch_type = school_data_full[['school_name','type']].drop_duplicates().set_index('school_name')

#student count by school
sch_stu_ct = school_data_full.groupby('school_name').count()['Student ID']

#school budget
sch_bdg = school_data_full.groupby('school_name').mean()['budget']

#used .rename to assign name to Series, otherwise won't be able to aggregate into summary table 
sch_stu_budg = (sch_bdg/sch_stu_ct).rename('Budget Per Student')

#average math score per school
sch_avg_math = round(school_data_full.groupby('school_name').mean()['math_score'],2)

#average reading score per school
sch_avg_read = round(school_data_full.groupby('school_name').mean()['reading_score'],2)

#percent of students passing math
mask2 = (school_data_full.math_score >=70)
sch_pass_math = school_data_full.school_name.loc[mask2].value_counts()
sch_pass_math_pct = (sch_pass_math/sch_stu_ct*100).rename('% Passing Math')

#percent of students passing reading
mask3 = (school_data_full.reading_score >=70)
sch_pass_read = school_data_full.school_name.loc[mask3].value_counts()
sch_pass_read_pct = (sch_pass_read/sch_stu_ct*100).rename('% Passing Reading')

#percent of students passing both math and reading
sch_pass_both = school_data_full.school_name.loc[mask].value_counts()
sch_pass_both_pct = (sch_pass_both/sch_stu_ct*100).rename('% Passing Math and Reading')

#join
sch_pt1 = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(sch_type, sch_stu_ct, on = "school_name"),sch_bdg,
            on="school_name"),sch_stu_budg, on = "school_name"),sch_avg_math, 
            on = "school_name"),sch_avg_read, on = "school_name")

#join the ones where the index has no name
sch_pt2 = pd.merge(pd.merge(sch_pass_math_pct, sch_pass_read_pct,left_index = True, right_index=True),sch_pass_both_pct,
         left_index = True, right_index=True)

#one happy family
sch_summary = pd.merge(sch_pt1,sch_pt2, left_index = True, right_index=True).rename(columns = {'type':'Type',
                'budget':'Budget','math_score':'Average Math Score','reading_score':'Average Reading Score'})

sch_summary_fmt = sch_summary.copy()
sch_summary_fmt['Budget'] = sch_summary_fmt['Budget'].map("${:,.2f}".format)
sch_summary_fmt['Budget Per Student'] = sch_summary_fmt['Budget Per Student'].map("${:,.2f}".format)
sch_summary_fmt['% Passing Math'] = sch_summary_fmt['% Passing Math'].map("{:.2f}%".format)
sch_summary_fmt['% Passing Reading'] = sch_summary_fmt['% Passing Reading'].map("{:.2f}%".format)
sch_summary_fmt['% Passing Math and Reading'] = sch_summary_fmt['% Passing Math and Reading'].map("{:.2f}%".format)

sch_summary_fmt

,Type,Student ID,Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Math and Reading
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.36,83.73,93.87%,95.85%,89.89%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

In [5]:
top_five = pd.DataFrame(round(sch_summary['% Passing Math and Reading'].sort_values(ascending = False).head(5),2))
top_five

,% Passing Math and Reading
Cabrera High School,91.33
Thomas High School,90.95
Griffin High School,90.60
Wilson High School,90.58
Pena High School,90.54


## Bottom Performing Schools (By % Overall Passing)

In [6]:
bottom_five = pd.DataFrame(round(sch_summary['% Passing Math and Reading'].sort_values().head(5),2))
bottom_five

,% Passing Math and Reading
Rodriguez High School,52.99
Figueroa High School,53.20
Huang High School,53.51
Hernandez High School,53.53
Johnson High School,53.54


## Reading Scores by Grade

In [7]:
#pull values for 9th grade via mask and then locate where true and average out scores
mask4 = [x for x in school_data_full.grade == '9th']
ninth_r = round(school_data_full[['school_name','reading_score']].loc[mask4].groupby('school_name').mean(),2)

#pull values for 10th grade via mask and then locate where true and average out scores
mask5 = [x for x in school_data_full.grade == '10th']
tenth_r = round(school_data_full[['school_name','reading_score']].loc[mask5].groupby('school_name').mean(),2)

#pull values for 11th grade via mask and then locate where true and average out scores
mask6 = [x for x in school_data_full.grade == '11th']
eleventh_r = round(school_data_full[['school_name','reading_score']].loc[mask6].groupby('school_name').mean(),2)

#pull values for 12th grade via mask and then locate where true and average out scores
mask7 = [x for x in school_data_full.grade == '12th']
twelfth_r = round(school_data_full[['school_name','reading_score']].loc[mask7].groupby('school_name').mean(),2)

#merge and assign column headers
grade_summary_r = pd.concat([ninth_r,tenth_r,eleventh_r,twelfth_r],axis=1)
grade_summary_r.columns = ['9th Grade', '10th Grade', '11th Grade','12th Grade']

grade_summary_r

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Math Score by Grade 

In [8]:
#locate corresponding grade columns and average scores - can reuse masks from reading
ninth_m = round(school_data_full[['school_name','math_score']].loc[mask4].groupby('school_name').mean(),2)
tenth_m = round(school_data_full[['school_name','math_score']].loc[mask5].groupby('school_name').mean(),2)
eleventh_m = round(school_data_full[['school_name','math_score']].loc[mask6].groupby('school_name').mean(),2)
twelfth_m = round(school_data_full[['school_name','math_score']].loc[mask7].groupby('school_name').mean(),2)

#merge and assign column headers
grade_summary_m = pd.concat([ninth_m,tenth_m,eleventh_m,twelfth_m],axis=1)
grade_summary_m.columns = ['9th Grade', '10th Grade', '11th Grade','12th Grade']

grade_summary_m

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Scores by School Spending

In [23]:
bins_spend = [570,590,610,630,660]
labels_spend = ['570-590', '591-610','611-630','631-660']

#bin the data and append
binned_spend = pd.cut(sch_summary['Budget Per Student'],bins_spend,labels = labels_spend)
sch_spend_bn = sch_summary.assign(Per_Student_Spending = binned_spend)
sch_spend_summary = round(sch_spend_bn.groupby('Per_Student_Spending').mean().drop(columns = ['Student ID',
                        'Budget','Budget Per Student']),2)
#format
sch_spend_summary['% Passing Math'] = sch_spend_summary['% Passing Math'].map("{:.2f}%".format)
sch_spend_summary['% Passing Reading'] = sch_spend_summary['% Passing Reading'].map("{:.2f}%".format)
sch_spend_summary['% Passing Math and Reading'] = sch_spend_summary['% Passing Math and Reading'].map(
    "{:.2f}%".format)

sch_spend_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Math and Reading
Per_Student_Spending,,,,,
570-590,83.45,83.94,93.46%,96.61%,90.37%
591-610,83.60,83.88,94.23%,95.90%,90.22%
611-630,80.20,82.42,80.04%,89.54%,72.62%
631-660,77.87,81.37,70.35%,83.00%,58.86%


## Scores by School Size

In [ ]:
#create bins and labels
bins_size = [0,1000,2000,5000]
labels_size = ['Small (<1000)', 'Medium (1000-2000)','Large (2000-5000)']

#bin data and append column to dataset
binned_size= pd.cut(school_data_full['size'],bins_size,labels = labels_size)
sch_size_bn = school_data_full.assign(Binned = binned_size)

#aggregate and drop columns to get average scores 
sch_size_avg = round(sch_size_bn.groupby('Binned').mean().drop(columns = ['Student ID','School ID','size']),2)

#get percent of students who passed THEN group by bin for math
mask10 = (sch_size_bn.math_score >=70)
math_pass_size = (sch_size_bn.Binned.loc[mask10].value_counts()/sch_size_bn.groupby(
    'Binned').count()['Student ID']*100).rename('% Passing Math')

#get percent of students who passed THEN group by bin for reading
mask11 = (sch_size_bn.reading_score >=70)
read_pass_size = (sch_size_bn.Binned.loc[mask11].value_counts()/sch_size_bn.groupby(
    'Binned').count()['Student ID']*100).rename('% Passing Reading')

#'' for both 
mask12 = (sch_size_bn.reading_score >=70) & (sch_size_bn.math_score >=70)
both_pass_size = (sch_size_bn.Binned.loc[mask12].value_counts()/sch_size_bn.groupby(
    'Binned').count()['Student ID']*100).rename('% Passing Math and Reading')
#merge to create summary table 
size_summary_raw = pd.merge(pd.merge(pd.merge(math_pass_size,read_pass_size, left_index = True, right_index=True),
                    both_pass_size,left_index = True, right_index=True),sch_size_avg,left_index = True, 
                    right_index=True)
size_summary = size_summary_raw.rename(columns = {'reading_score':'Average Reading Score','math_score':
                    'Average Math Score','budget':'Average Budget'})

#format
size_summary['Average Budget'] = size_summary['Average Budget'].map("${:,.2f}".format)
size_summary['% Passing Math'] = size_summary['% Passing Math'].map("{:.2f}%".format)
size_summary['% Passing Reading'] = size_summary['% Passing Reading'].map("{:.2f}%".format)
size_summary['% Passing Math and Reading'] = size_summary['% Passing Math and Reading'].map("{:.2f}%".format)

size_summary

## Scores by School Type

In [ ]:
#average scores, size, and budget  
type_summary = round(sch_summary.groupby('Type').mean().drop(columns = ['Student ID']),2)

#format
type_summary['Budget'] = type_summary['Budget'].map("${:,.2f}".format)
type_summary['Budget Per Student'] = type_summary['Budget Per Student'].map("${:,.2f}".format)
type_summary['% Passing Math'] = type_summary['% Passing Math'].map("{:.2f}%".format)
type_summary['% Passing Reading'] = type_summary['% Passing Reading'].map("{:.2f}%".format)
type_summary['% Passing Math and Reading'] = type_summary['% Passing Math and Reading'].map("{:.2f}%".format)

type_summary

In [ ]:
#would also be interesting to see how avg funding differs from charter vs district schools 
#both in total and per student 

In [ ]:
#would be nice to have data on teacher:student - does having a higher ratio correlate with better test scores?
#also would be nice to have info on median income in the district - low SES impacts success outcomes w/ education